In [16]:
import numpy as np
import pandas as pd
import xgboost as xgb

from boostaroota import BoostARoota
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [12]:
def prepare_data(X: pd.DataFrame) -> pd.DataFrame:
    """
    Подготовка набора данных для моделирования.
    Категориальные признаки преобразуются с помощью
    OneHotEnconding.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков.

    Returns
    -------
    X_transformed: pandas.core.frame.DataFrame
        Преобразованная матрица признаков.

    """
    categorical = X.dtypes[X.dtypes=="object"].index.tolist()

    X = pd.get_dummies(X, columns=categorical)
    X.columns = [col.lower() for col in X.columns]
    
    return X

In [15]:
data = pd.read_csv(
    "../data/competition_data/train.csv"
)
features = pd.read_csv(
    "../data/competition_data/client_profile.csv"
)
data = data.merge(
    features, how="inner", on="APPLICATION_NUMBER"
)
data = prepare_data(data)

In [18]:
x_train, x_valid = train_test_split(
    data.drop(["target", "application_number"], axis=1), train_size=0.7, random_state=27
)
y_train, y_valid = train_test_split(
    data["target"], train_size=0.7, random_state=27
)

## Base XGBoost

In [19]:
params = {
    "eta": 0.01,
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(y_train),
    "eval_metric": "auc"
}

dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [20]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.722133	valid-auc:0.690937
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.772349	valid-auc:0.735562
[200]	train-auc:0.787935	valid-auc:0.740267
[300]	train-auc:0.801033	valid-auc:0.74283
[400]	train-auc:0.811698	valid-auc:0.744505
[500]	train-auc:0.819985	valid-auc:0.744964
Stopping. Best iteration:
[546]	train-auc:0.824612	valid-auc:0.745207



## Base BoostARoota

__Fast XGBoost Feature Selection Algorithm, как это работает?__

How it works

Similar in spirit to Boruta, BoostARoota creates shadow features, but modifies the removal step.

* `One-Hot-Encode` признаки;

* Дублировать каждый признак в исходном наборе данных => признаков в 2 раза больше;

* Случайным образом перемешать новые признаки, созданные в п.2. Эти продублированные и перемешанные признаки называются «теневыми признаками» (`shadow features`);

* Запустить классификатор `XGBoost` для всего набора данных десять раз. Запуск его десять раз позволяет сгладить случайный шум, что приводит к более надежным оценкам важности. Количество повторов - это гиперпараметр, который можно изменить.

* Получить значения важности для каждого призанка. Это простой показатель важности, который суммирует количество сплитов, в которых участвовал данный признак.

* Вычислитm «отсечку»: среднее значение важности для всех теневых признаков и разделить на четыре. Значения теневой важности делятся на четыре (параметр можно изменить), чтобы было сложнее удалить признаки. При значениях ниже этого значения, признаки удаляются.

* Удалить признаки со средней важностью за десять итераций, которая меньше порогового значения, указанного в п.6ю

* Вернуться к п.2, пока количество удаленных признаков не станет менее десяти процентов от общего числа.

In [61]:
br = BoostARoota(metric='auc')
br.fit(x_train, y_train)

Round:  1  iteration:  1
Round:  1  iteration:  2
Round:  1  iteration:  3
Round:  1  iteration:  4
Round:  1  iteration:  5
Round:  1  iteration:  6
Round:  1  iteration:  7
Round:  1  iteration:  8
Round:  1  iteration:  9
Round:  1  iteration:  10
Round:  2  iteration:  1
Round:  2  iteration:  2
Round:  2  iteration:  3
Round:  2  iteration:  4
Round:  2  iteration:  5
Round:  2  iteration:  6
Round:  2  iteration:  7
Round:  2  iteration:  8
Round:  2  iteration:  9
Round:  2  iteration:  10
Round:  3  iteration:  1
Round:  3  iteration:  2
Round:  3  iteration:  3
Round:  3  iteration:  4
Round:  3  iteration:  5
Round:  3  iteration:  6
Round:  3  iteration:  7
Round:  3  iteration:  8
Round:  3  iteration:  9
Round:  3  iteration:  10
BoostARoota ran successfully! Algorithm went through  3  rounds.


In [62]:
br.keep_vars_

0                                         childrens
1                                      total_salary
2                                     amount_credit
3                                    amount_annuity
4                                 region_population
5                                               age
6                                  days_on_last_job
7                                       own_car_age
8                                       family_size
9                         external_scoring_rating_1
10                        external_scoring_rating_2
11                        external_scoring_rating_3
12                       amt_req_credit_bureau_hour
14                        amt_req_credit_bureau_mon
15                        amt_req_credit_bureau_qrt
16                       amt_req_credit_bureau_year
17                          name_contract_type_cash
18                                         gender_f
19                                         gender_m
20          

In [23]:
x_train_br = br.transform(x_train)
x_valid_br = br.transform(x_valid)

dtrain_br = xgb.DMatrix(x_train_br, label=y_train)
dvalid_br = xgb.DMatrix(x_valid_br, label=y_valid)

In [24]:
model_br = xgb.train(
    params,
    dtrain_br,
    num_boost_round=5000,
    evals=[(dtrain_br, "train"), (dvalid_br, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.722133	valid-auc:0.690937
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.772295	valid-auc:0.735525
[200]	train-auc:0.787503	valid-auc:0.740233
[300]	train-auc:0.800326	valid-auc:0.742654
[400]	train-auc:0.811072	valid-auc:0.744297
[500]	train-auc:0.819349	valid-auc:0.74509
Stopping. Best iteration:
[546]	train-auc:0.823974	valid-auc:0.745396



In [25]:
old_preds = model.predict(dvalid)
new_preds = model_br.predict(dvalid_br)

old_score = roc_auc_score(y_valid, old_preds)
new_score = roc_auc_score(y_valid, new_preds)
print(f"Base XGBoost score = {round(old_score, 5)}")
print(f"Base BoostARoota score = {round(new_score, 5)}")

Base XGBoost score = 0.74509
Base BoostARoota score = 0.74532


## Optimize BoostARoota

* __`clf [default = None]`__ - опциональный параметр, рекомендуется оставить пустым;

    * По умолчанию будет `xgboost`, если оставить пустым
    * Можно использовать любую древовидную модель из `sklearn`

* __`cutoff [default = 4]`__ - порог отсечки

    * Отсечка для отбора признаков
    * Большие значения работают более консервативно - итоговый набор признаков будет достаточно большим, удаляется мало признаков. Маленькие значения работают более агрессивно;

* __`iters [default = 10]`__ - количество итераций для оценки важности признаков;

    * Маленькие значения позволяет получить результат быстрее, потому что `XGBoost` обучается меньшее число раз.
    * Масштабируется линейно: `iters` = 4 требует в 2 раза больше времени чем `iters` = 2 и в 4 раза больше, чем `iters` = 1.
    
* __`max_rounds [default = 100]`__ - количество итераций алгоритма `BoostARoota`. Каждый запуск исключает все больше и больше признаков. 

In [28]:
br = BoostARoota(metric='auc', iters=50)
br.fit(x_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Round:  1  iteration:  1
Round:  1  iteration:  2
Round:  1  iteration:  3
Round:  1  iteration:  4
Round:  1  iteration:  5
Round:  1  iteration:  6
Round:  1  iteration:  7
Round:  1  iteration:  8
Round:  1  iteration:  9
Round:  1  iteration:  10
Round:  1  iteration:  11
Round:  1  iteration:  12
Round:  1  iteration:  13
Round:  1  iteration:  14
Round:  1  iteration:  15
Round:  1  iteration:  16
Round:  1  iteration:  17
Round:  1  iteration:  18
Round:  1  iteration:  19
Round:  1  iteration:  20
Round:  1  iteration:  21
Round:  1  iteration:  22
Round:  1  iteration:  23
Round:  1  iteration:  24
Round:  1  iteration:  25
Round:  1  iteration:  26
Round:  1  iteration:  27
Round:  1  iteration:  28
Round:  1  iteration:  29
Round:  1  iteration:  30
Round:  1  iteration:  31
Round:  1  iteration:  32
Round:  1  iteration:  33
Round:  1  iteration:  34
Round:  1  iteration:  35
Round:  1  iteration:  36
Round:  1  iteration:  37
Round:  1  iteration:  38
Round:  1  iteration:

In [29]:
x_train_br = br.transform(x_train)
x_valid_br = br.transform(x_valid)

dtrain_br = xgb.DMatrix(x_train_br, label=y_train)
dvalid_br = xgb.DMatrix(x_valid_br, label=y_valid)

In [30]:
model_br = xgb.train(
    params,
    dtrain_br,
    num_boost_round=5000,
    evals=[(dtrain_br, "train"), (dvalid_br, "valid")],
    early_stopping_rounds=50,
    verbose_eval=100
)

[0]	train-auc:0.722133	valid-auc:0.690937
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[100]	train-auc:0.772088	valid-auc:0.73577
[200]	train-auc:0.787172	valid-auc:0.740577
[300]	train-auc:0.799878	valid-auc:0.743006
[400]	train-auc:0.81069	valid-auc:0.744628
[500]	train-auc:0.818963	valid-auc:0.745408
[600]	train-auc:0.828223	valid-auc:0.745712
[700]	train-auc:0.836154	valid-auc:0.745864
Stopping. Best iteration:
[673]	train-auc:0.833884	valid-auc:0.745982



In [31]:
old_preds = model.predict(dvalid)
new_preds = model_br.predict(dvalid_br)

old_score = roc_auc_score(y_valid, old_preds)
new_score = roc_auc_score(y_valid, new_preds)
print(f"Base XGBoost score = {round(old_score, 5)}")
print(f"Base BoostARoota score = {round(new_score, 5)}")

Base XGBoost score = 0.74509
Base BoostARoota score = 0.74587
